TPGshipのシミュレーション動画作成プログラム

ライブラリ読み込み

In [9]:
import cartopy.crs as ccrs
import os
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scienceplots
import polars as pl
import numpy as np
import math
from numpy import linalg as LA
import cartopy.feature as cfeature
from PIL import Image
import matplotlib.patches as patches
from datetime import datetime, timedelta, timezone
from dateutil import tz
import sys


データファイル読み取り

台風＆TPGship経路データ

In [10]:
year = 2018

ship_typhoon_route_data = pl.read_csv("TPGshipdata_ver5/" + str(year) + "_sim_ver5.csv",encoding="shift-jis")

typhoon_data = pl.read_csv("typhoon_data_6hour_intervals_verpl/table" + str(year) + "_6_interval.csv",encoding="shift-jis")

拠点と運搬船

In [4]:
stBASE_data = pl.read_csv("storageBASEdata_ver5/" + str(year) + "storageBASE.csv")
spSHIP1_data = pl.read_csv("supportSHIPdata_ver5/" + str(year) + "supportSHIP1.csv")
spSHIP2_data = pl.read_csv("supportSHIPdata_ver5/" + str(year) + "supportSHIP2.csv") 

TPGshipの動作画像化

In [4]:
UTC = timezone(timedelta(hours =+ 0),"UTC")

#データの整理
totalgene = ship_typhoon_route_data["YEARLY POWER GENERATION BALANCE"]
tg = []
for i in range(len(totalgene)):
    tg.append(totalgene[i]/10**9)

onboardene = ship_typhoon_route_data["ONBOARD ENERGY STORAGE[Wh]"]
obe = []
for i in range(len(onboardene)):
    obe.append(onboardene[i]/10**9)

day = ship_typhoon_route_data["unixtime"]
daylist = []
for i in range(len(day)):
    daylist.append((day[i]-day[0])/86400)

for j in range(len(ship_typhoon_route_data)):
#for j in range(4):
#    j_ori = j
#    j = j * 470
    #地図の作成
    fig = plt.figure(figsize=(25,16)) # プロット領域の作成（matplotlib）
    ncols = 2
    nrows = 2
    #gs = gridspec.GridSpec(nrows, ncols, width_ratios=[1, 1], height_ratios=[1, 1])

    
    ax = fig.add_subplot(121, projection=ccrs.PlateCarree()) # サブプロット作成時にcartopy呼び出し
    #ax =  fig.add_subplot(gs[:2, 0])
    

    ax.set_facecolor('paleturquoise')
    land_h = cfeature.NaturalEarthFeature('physical', 'land', '50m')
    ax.add_feature(land_h,color='g')
    ax.set_extent([120, 180, 0, 75], ccrs.Geodetic())

    

    #中継貯蔵拠点&待機位置
    base_lat = 24
    base_lon = 153
    ax.plot(base_lon,base_lat,'crimson',markersize=15,marker="d")

    #供給拠点
    spbase_lat = 34.75
    spbase_lon = 134.79
    ax.plot(spbase_lon,spbase_lat,'crimson',markersize=20,marker="*")



    #陸地境界の設定
    #nonaggression_line_lat = [0,13,13,15,15,24,24,26,26,28,28,32.2,32.2,34,34,41.2,41.2,44,44,50,50]
    #nonaggression_line_lon = [127.5,127.5,125,125,123,123,126,126,130.1,130.1,132.4,132.4,137.2,137.2,143,143,149,149,156,156,180]
    #for i in range(len(nonaggression_line_lat)-1):
    #    ax.plot([nonaggression_line_lon[i],nonaggression_line_lon[i+1]],[nonaggression_line_lat[i],nonaggression_line_lat[i+1]],'red',linewidth=4)

    #台風の作成 
    unixtime = ship_typhoon_route_data[j,"unixtime"]
    
    TYdata = typhoon_data.filter((pl.col("unixtime") == unixtime))
    typhoon_lon = TYdata[:]["LON"]
    typhoon_lat = TYdata[:]["LAT"]
    if len(typhoon_lon) != 0:
        for k in range(len(typhoon_lon)):
            ax.plot(typhoon_lon[k],typhoon_lat[k],'grey',markersize=50,marker="o",alpha=0.8)


    #凡例の作成
    view_lon = 170
    text_lon = 173
    d_view_lat = 3
    view_ship_lon = 150
    view_ship_lat = 60
    view_lat_1 = 58
    lon_width = 2

    ship_text_lon = 154
    ship_text_lat = 63.5

    #目標名の設定
    target = ship_typhoon_route_data[j,"TARGET LOCATION"]
    if ship_typhoon_route_data[j,"TARGET TYPHOON"] != 0:
        target = "Typhoon " + str(target)
    else:
        target = str(target)
    
    #船速取得
    speed = str(format(ship_typhoon_route_data[j,"SHIP SPEED[kt]"], '.1f'))

    #状態表示　移動・発電・待機
    if (ship_typhoon_route_data[j,"SHIP SPEED[kt]"] == 0)&(ship_typhoon_route_data[j,"TIMESTEP POWER GENERATION[Wh]"] == 0) & (ship_typhoon_route_data[j,"TIMESTEP POWER CONSUMPTION[Wh]"] == 0):
        state = "Standby"
        iro = "lime"
    elif ship_typhoon_route_data[j,"TIMESTEP POWER GENERATION[Wh]"] > 0:
        state = "Power Generation"
        speed = speed + "(prov)"
        iro = "yellow"
    elif (ship_typhoon_route_data[j,"TIMESTEP POWER CONSUMPTION[Wh]"] > 0) or (ship_typhoon_route_data[j,"SHIP SPEED[kt]"] > 0):
        state = "Moving"
        iro = "red"
    
    else:
        print("Error")

    #船内蓄電量
    storage_per = str(format(ship_typhoon_route_data[j,"ONBOARD POWER STORAGE PER[%]"], '.1f'))

    #TPGship関連情報表示
    r = patches.Rectangle(xy=(145, 58), width=40, height=13, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    

    ax.quiver(view_ship_lon, view_ship_lat, 0, 55, color=iro, edgecolor="k", linewidth = 1.0, headwidth=30, headlength=50 , headaxislength=50, width=.02, scale=400.,zorder=4)
    ax.text(ship_text_lon, ship_text_lat+5.2, "TPGship prototype", size=25, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat+2.5, "Target          : "+target, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-0.0, "States          : "+state, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-2.5, "Speed[kt]    : "+speed, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-5.0, "Storage[\%]  : "+storage_per, size=18, color="black",zorder=4)


    #日数記録
    date = str(datetime.fromtimestamp(unixtime,UTC))
    r = patches.Rectangle(xy=(145, 54), width=24, height=4, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    ax.text(145.5, view_lat_1 - 0.8*d_view_lat-0.4, date, size=16.5, color="black",zorder=4)

    #台風の凡例表示
    r = patches.Rectangle(xy=(167, 54), width=20, height=4, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    
    ax.plot(view_lon,view_lat_1 - 0.7*d_view_lat,'grey',markersize=30,marker="o",zorder=4)
    ax.text(text_lon, view_lat_1 - 0.7*d_view_lat-0.5, "Typhoon", size=15, color="black",zorder=4)

    #中継貯蔵拠点の凡例表示
    r = patches.Rectangle(xy=(145, 46), width=20, height=8, ec='k', fc='w',zorder=3)
    stbase_state = str(stBASE_data[j,"BRANCH CONDITION"])
    stbase_storage = str(format(stBASE_data[j,"STORAGE PER[%]"], '.1f'))
    ax.add_patch(r)
    ax.plot(147.0, view_lat_1 - 2.4*d_view_lat,'crimson',markersize=30,marker="d",zorder=4)
    ax.text(149.0, view_lat_1 - 1.8*d_view_lat-0.5, "Storage Base", size=20, color="black",zorder=4)
    ax.text(149.0, view_lat_1 - 2.3*d_view_lat-0.5, "States : "+stbase_state, size=14, color="black",zorder=4)
    ax.text(149.0, view_lat_1 - 2.8*d_view_lat-0.5, "Storage[%]  : "+stbase_storage, size=14, color="black",zorder=4)

    #供給拠点の凡例表示
    ax.plot(147.0, view_lat_1 - 3.6*d_view_lat,'crimson',markersize=20,marker="*",zorder=4)
    ax.text(149.0, view_lat_1 - 3.6*d_view_lat-0.5, "Supply Base", size=15, color="black",zorder=4)

    #運搬船の凡例表示
    r = patches.Rectangle(xy=(165, 46), width=20, height=8, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    spship1_storage = str(format(spSHIP1_data[j,"STORAGE PER[%]"], '.1f'))
    spship2_storage = str(format(spSHIP2_data[j,"STORAGE PER[%]"], '.1f'))
    ax.quiver(166.5, 51, 0, 15, color="navy", edgecolor="k", linewidth = 0.25, headwidth=6, headlength=10 , headaxislength=10, width=.02, scale=400.,zorder=4)
    ax.text(168.0, view_lat_1 - 1.7*d_view_lat-0.5, "SupportSHIP1", size=15, color="black",zorder=4)
    ax.text(168.0, view_lat_1 - 2.2*d_view_lat-0.5, "Storage[\%]  : "+spship1_storage, size=13, color="black",zorder=4)
    ax.quiver(166.5, 46.7, 0, 15, color="purple", edgecolor="k", linewidth = 0.25, headwidth=6, headlength=10 , headaxislength=10, width=.02, scale=400.,zorder=4)
    ax.text(168.0, view_lat_1 - 3.1*d_view_lat-0.5, "SupportSHIP2", size=15, color="black",zorder=4)
    ax.text(168.0, view_lat_1 - 3.6*d_view_lat-0.5, "Storage[\%]  : "+spship2_storage, size=13, color="black",zorder=4)


    #運搬船1の作図
    spship1_lat = spSHIP1_data[j,"LAT"]
    spship1_lon = spSHIP1_data[j,"LON"]
    if j == 0:
        u = 0
        v = 1
    else:
        spship1_lat_2 = spSHIP1_data[j-1,"LAT"]
        spship1_lon_2 = spSHIP1_data[j-1,"LON"]
        u = (spship1_lon - spship1_lon_2)
        v = (spship1_lat - spship1_lat_2)
        if (u == 0) & (v == 0):
            u = 0
            v = 1


    size = 10
    
    vec_size = (np.sqrt(u**2 + v**2))
    ax.quiver(spship1_lon, spship1_lat, u/vec_size*size, v/vec_size*size, color="navy", edgecolor="k", linewidth = 0.25, headlength=6 , headaxislength=6, width=.004, scale=400.,zorder=4.5)
    
    #運搬船2の作図
    spship2_lat = spSHIP2_data[j,"LAT"]
    spship2_lon = spSHIP2_data[j,"LON"]
    if j == 0:
        u = 0
        v = 1
    else:
        spship2_lat_2 = spSHIP2_data[j-1,"LAT"]
        spship2_lon_2 = spSHIP2_data[j-1,"LON"]
        u = (spship2_lon - spship2_lon_2)
        v = (spship2_lat - spship2_lat_2)
        if (u == 0) & (v == 0):
            u = 0
            v = 1


    size = 10
    
    vec_size = (np.sqrt(u**2 + v**2))
    ax.quiver(spship2_lon, spship2_lat, u/vec_size*size, v/vec_size*size, color="purple", edgecolor="k", linewidth = 0.25, headlength=6 , headaxislength=6, width=.004, scale=400.,zorder=4.5)
    



    #TPGshipの作図
    ship_lat = ship_typhoon_route_data[j,"TPGSHIP LAT"]
    ship_lon = ship_typhoon_route_data[j,"TPGSHIP LON"]
    if j == 0:
        u = 0
        v = 1
    else:
        ship_lat_2 = ship_typhoon_route_data[j-1,"TPGSHIP LAT"]
        ship_lon_2 = ship_typhoon_route_data[j-1,"TPGSHIP LON"]
        u = (ship_lon - ship_lon_2)
        v = (ship_lat - ship_lat_2)
        if (u == 0) & (v == 0):
            u = 0
            v = 1


    size = 10
    
    vec_size = (np.sqrt(u**2 + v**2))
    ax.quiver(ship_lon, ship_lat, u/vec_size*size, v/vec_size*size, color=iro, edgecolor="k", linewidth = 0.25, headlength=6 , headaxislength=6, width=.004, scale=400.,zorder=5)
    
    #グラフの表示
    #今回は'science'と'ieee'を利用してみる
    plt.style.use(['science','ieee','high-vis','grid'])
    plt.rcParams["font.size"] = 20

    day_x = daylist[0:j]
    obe_y = obe[0:j]
    tg_y = tg[0:j]

    ax1 =  fig.add_subplot(222)
    ax1.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax1.set_ylim(ymin=0,ymax=75) #y軸の範囲を指定
    ax1.plot(day_x,obe_y,label = 'ONBOARD', linewidth=3)
    ax1.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax1.set(ylabel = 'ONBOARD POWER STORAGE[GWh]') #y軸のラベル
    #ax1.tick_params(labelsize=18)
    
    ax2 =  fig.add_subplot(224)
    ax2.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax2.set_ylim(ymin=0,ymax=350) #y軸の範囲を指定
    ax2.plot(day_x,tg_y,label = 'TOTAL', linewidth=3)
    ax2.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax2.set(ylabel = 'Power Generation[GWh]') #y軸のラベル
    #plt.tick_params(labelsize=18)

    

    #j = j_ori
    
    plt.savefig("png_Folder/draw" + str(j) + ".png")
    plt.close(fig)

    #im = Image.open("png_Folder/draw" + str(j) + ".png")

    #im_crop = im.crop((0, 0, 11500, 8000))
    
    #im_crop.save("png_Folder/draw" + str(j) + ".png", quality=100)
    


/opt/conda/lib/python3.10/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_physical/ne_50m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


: 

In [6]:
import gc

# グラフ保存用のフォルダがなければ作成
os.makedirs("png_Folder1", exist_ok=True)

for j in range(len(ship_typhoon_route_data)):
#for j in range(4):
#    j_ori = j
#    j = j + 540
    #地図の作成
    fig = plt.figure(figsize=(25,16)) # プロット領域の作成（matplotlib）
    ncols = 1
    nrows = 5
    #gs = gridspec.GridSpec(nrows, ncols, width_ratios=[1, 1], height_ratios=[1, 1])

    sim_fig = Image.open("png_Folder2/draw" + str(j) + ".png")
    
    ax = fig.add_subplot(1,5,(1,3))
    ax.imshow(sim_fig)
    ax.axis("off")

    graf_fig = Image.open("png_Folder3/draw" + str(j) + ".png")

    ax1 = fig.add_subplot(1,5,(4,5))
    ax1.imshow(graf_fig)
    ax1.axis("off")

    plt.savefig("png_Folder1/draw" + str(j) + ".png")
    plt.close(fig)

    im = Image.open("png_Folder1/draw" + str(j) + ".png")

    im_crop = im.crop((360, 180, 2260, 1440))
    
    im_crop.save("png_Folder1/draw" + str(j) + ".png", quality=100)
    
    # モジュールのキャッシュをクリアする
    gc.collect()



: 

In [5]:
UTC = timezone(timedelta(hours =+ 0),"UTC")

#データの整理
totalgene = ship_typhoon_route_data["YEARLY POWER GENERATION BALANCE"]
tg = []
for i in range(len(totalgene)):
    tg.append(totalgene[i]/10**9)

onboardene = ship_typhoon_route_data["ONBOARD ENERGY STORAGE[Wh]"]
obe = []
for i in range(len(onboardene)):
    obe.append(onboardene[i]/10**9)

basestorage = stBASE_data["STORAGE[Wh]"]
base_data = []
for i in range(len(basestorage)):
    base_data.append(basestorage[i]/10**9)

day = ship_typhoon_route_data["unixtime"]
daylist = []
for i in range(len(day)):
    daylist.append((day[i]-day[0])/86400)

# グラフ保存用のフォルダがなければ作成
os.makedirs("png_Folder3", exist_ok=True)

#for j in range(len(ship_typhoon_route_data)):
for j in range(2):
#    j_ori = j
#    j = j * 470
    #地図の作成
    fig = plt.figure(figsize=(10,16)) # プロット領域の作成（matplotlib）
    ncols = 3
    nrows = 1

    #グラフの表示
    #今回は'science'と'ieee'を利用してみる
    plt.style.use(['science','ieee','high-vis','grid'])
    plt.rcParams["font.size"] = 20

    day_x = daylist[0:j]
    obe_y = obe[0:j]
    tg_y = tg[0:j]
    base_data_y = base_data[0:j]

    ax1 =  fig.add_subplot(3,1,1)
    ax1.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax1.set_ylim(ymin=0,ymax=75) #y軸の範囲を指定
    ax1.plot(day_x,obe_y,label = 'ONBOARD', linewidth=3)
    ax1.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax1.set(ylabel = 'Onboard Power Storage[GWh]') #y軸のラベル
    #ax1.tick_params(labelsize=18)
    
    ax2 =  fig.add_subplot(3,1,3)
    ax2.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax2.set_ylim(ymin=0,ymax=350) #y軸の範囲を指定
    ax2.plot(day_x,tg_y,label = 'TOTAL', linewidth=3)
    ax2.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax2.set(ylabel = 'Power Generation[GWh]') #y軸のラベル
    #plt.tick_params(labelsize=18)

    ax3 =  fig.add_subplot(3,1,2)
    ax3.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax3.set_ylim(ymin=0,ymax=100) #y軸の範囲を指定
    ax3.plot(day_x,base_data_y,label = 'BASE OPERATION', linewidth=3)
    ax3.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax3.set(ylabel = 'Electric at Storage Base[GWh]') #y軸のラベル

    #j = j_ori
    
    plt.savefig("png_Folder3/draw" + str(j) + ".png")
    plt.close(fig)
    

: 

In [5]:
#Chat-GPTによる改善提案コード
import gc
import os

UTC = timezone(timedelta(hours=0), "UTC")

# データの整理
totalgene = (val / 10**9 for val in ship_typhoon_route_data["YEARLY POWER GENERATION BALANCE"])
onboardene = (val / 10**9 for val in ship_typhoon_route_data["ONBOARD ENERGY STORAGE[Wh]"])
basestorage = (val / 10**9 for val in stBASE_data["STORAGE[Wh]"])
day = ship_typhoon_route_data["unixtime"]
daylist = ((val - day[0]) / 86400 for val in day)

# グラフ保存用のフォルダがなければ作成
os.makedirs("png_Folder3", exist_ok=True)



for j, (total, onboard, base, timestamp) in enumerate(zip(totalgene, onboardene, basestorage, daylist)):
 
    fig = plt.figure(figsize=(10, 16))
    ncols = 3
    nrows = 1

    plt.style.use(['science', 'ieee', 'high-vis', 'grid'])
    plt.rcParams["font.size"] = 20

    day_x = []
    obe_y = []
    tg_y = []
    base_data_y = []

    for i in range(j + 1):
        day_x.append(timestamp)
        obe_y.append(onboard)
        tg_y.append(total)
        base_data_y.append(base)

    ax1 = fig.add_subplot(3, 1, 1)
    ax1.set_xlim(xmin=0, xmax=365)
    ax1.set_ylim(ymin=0, ymax=75)
    ax1.plot(day_x, obe_y, label='ONBOARD', linewidth=3)
    ax1.set(xlabel='Operation Time[Day]')
    ax1.set(ylabel='Onboard Power Storage[GWh]')

    ax2 = fig.add_subplot(3, 1, 3)
    ax2.set_xlim(xmin=0, xmax=365)
    ax2.set_ylim(ymin=0, ymax=350)
    ax2.plot(day_x, tg_y, label='TOTAL', linewidth=3)
    ax2.set(xlabel='Operation Time[Day]')
    ax2.set(ylabel='Power Generation[GWh]')

    ax3 = fig.add_subplot(3, 1, 2)
    ax3.set_xlim(xmin=0, xmax=365)
    ax3.set_ylim(ymin=0, ymax=100)
    ax3.plot(day_x, base_data_y, label='BASE OPERATION', linewidth=3)
    ax3.set(xlabel='Operation Time[Day]')
    ax3.set(ylabel='Electric at Storage Base[GWh]')

    # メモリ解放のためのガベージコレクションの呼び出し
    plt.savefig("png_Folder3/draw" + str(j) + ".png")
    plt.close(fig)
    gc.collect()


: 

In [ ]:
UTC = timezone(timedelta(hours =+ 0),"UTC")

#データの整理
totalgene = ship_typhoon_route_data["YEARLY POWER GENERATION BALANCE"]
tg = []
for i in range(len(totalgene)):
    tg.append(totalgene[i]/10**9)

onboardene = ship_typhoon_route_data["ONBOARD ENERGY STORAGE[Wh]"]
obe = []
for i in range(len(onboardene)):
    obe.append(onboardene[i]/10**9)

basestorage = stBASE_data["STORAGE[Wh]"]
base_data = []
for i in range(len(basestorage)):
    base_data.append(basestorage[i]/10**9)

day = ship_typhoon_route_data["unixtime"]
daylist = []
for i in range(len(day)):
    daylist.append((day[i]-day[0])/86400)


# グラフ保存用のフォルダがなければ作成
os.makedirs("png_Folder3", exist_ok=True)

#for j in range(len(ship_typhoon_route_data)):
for j in range(4):
#    j_ori = j
    j = j * 470
    #地図の作成
    fig = plt.figure(figsize=(25,16)) # プロット領域の作成（matplotlib）
    ncols = 3
    nrows = 5
    #gs = gridspec.GridSpec(nrows, ncols, width_ratios=[1, 1], height_ratios=[1, 1])

    sim_fig = Image.open("png_Folder2/draw" + str(j) + ".png")
    ax = fig.add_subplot(1,5,(1,3))
    ax.imshow(sim_fig)
    ax.axis("off")

    #グラフの表示
    #今回は'science'と'ieee'を利用してみる
    plt.style.use(['science','ieee','high-vis','grid'])
    plt.rcParams["font.size"] = 20

    day_x = daylist[0:j]
    obe_y = obe[0:j]
    tg_y = tg[0:j]
    base_data_y = base_data[0:j]

    ax1 =  fig.add_subplot(3,5,(4,5))
    ax1.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax1.set_ylim(ymin=0,ymax=75) #y軸の範囲を指定
    ax1.plot(day_x,obe_y,label = 'ONBOARD', linewidth=3)
    ax1.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax1.set(ylabel = 'ONBOARD STORAGE[GWh]') #y軸のラベル
    #ax1.tick_params(labelsize=18)
    
    ax2 =  fig.add_subplot(3,5,(14,15))
    ax2.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax2.set_ylim(ymin=0,ymax=350) #y軸の範囲を指定
    ax2.plot(day_x,tg_y,label = 'TOTAL', linewidth=3)
    ax2.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax2.set(ylabel = 'Power Generation[GWh]') #y軸のラベル
    #plt.tick_params(labelsize=18)

    ax3 =  fig.add_subplot(3,5,(9,10))
    ax3.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax3.set_ylim(ymin=0,ymax=100) #y軸の範囲を指定
    ax3.plot(day_x,base_data_y,label = 'BASE OPERATION', linewidth=3)
    ax3.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax3.set(ylabel = 'Electric at Storage Base[GWh]') #y軸のラベル

    #j = j_ori
    
    plt.savefig("png_Folder3/draw" + str(j) + ".png")
    plt.close(fig)

    #im = Image.open("png_Folder/draw" + str(j) + ".png")

    #im_crop = im.crop((0, 0, 11500, 8000))
    
    #im_crop.save("png_Folder/draw" + str(j) + ".png", quality=100)
    


: 

In [3]:
print(len(ship_typhoon_route_data))

1461


動画化

In [12]:
import cv2
import sys

year = 2018
# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# output file name, encoder, fps, size(fit to image size)
video = cv2.VideoWriter(str(year) + '_map_data_ver5_32.mp4',fourcc, 32.0, (670, 800))#サイズ揃える

if not video.isOpened():
    print("can't be opened")
    sys.exit()

for i in range(len(ship_typhoon_route_data)):
    img = cv2.imread('png_Folder/draw'+str(i)+'.png')


    # can't read image, escape
    if img is None:
        print("can't read")
        break
    

    # add
    video.write(img)

video.release()
print('written')

written
